In [101]:
import sys
sys.path.insert(1, '/Users/dabiyyu/Downloads/mlprocess/src')
import util as utils
import pandas as pd

In [102]:
config = utils.load_config()

# Load Dataset

In [103]:
def load_dataset(config_data: dict) -> pd.DataFrame:
    # Load every set of data
    X_train = utils.pickle_load(config_data["train_set_path"][0])
    y_train = utils.pickle_load(config_data["train_set_path"][1])

    X_valid = utils.pickle_load(config_data["valid_set_path"][0])
    y_valid = utils.pickle_load(config_data["valid_set_path"][1])

    X_test = utils.pickle_load(config_data["test_set_path"][0])
    y_test = utils.pickle_load(config_data["test_set_path"][1])

    # Concatenate x and y each set
    train_set = pd.concat([X_train, y_train], axis = 1)
    valid_set = pd.concat([X_valid, y_valid], axis = 1)
    test_set = pd.concat([X_test, y_test], axis = 1)
    
    # Return 3 set of data
    return train_set, valid_set, test_set

In [104]:
train_set, valid_set, test_set = load_dataset(config)

# Removing Outlier

In [105]:
def remove_outliers(set_data):
    set_data = set_data.copy()
    list_of_set_data = list()

    for col_name in set_data.columns[:-1]:
        q1 = set_data[col_name].quantile(0.25)
        q3 = set_data[col_name].quantile(0.75)
        iqr = q3 - q1
        set_data_cleaned = set_data[~((set_data[col_name] < (q1 - 1.5 * iqr)) | (set_data[col_name] > (q3 + 1.5 * iqr)))].copy()
        list_of_set_data.append(set_data_cleaned.copy())
    
    set_data_cleaned = pd.concat(list_of_set_data)
    count_duplicated_index = set_data_cleaned.index.value_counts()
    used_index_data = count_duplicated_index[count_duplicated_index == (set_data.shape[1]-1)].index
    set_data_cleaned = set_data_cleaned.loc[used_index_data].drop_duplicates()

    return set_data_cleaned

In [106]:
train_set_cleaned = remove_outliers(train_set)

In [107]:
train_set_cleaned

,AT,V,AP,RH,PE
8759,16.69,44.78,1010.41,58.18,456.17
9516,27.35,77.95,1017.43,73.36,437.61
4259,27.81,66.51,1016.11,48.48,439.48
8660,18.89,63.09,1018.47,84.85,450.00
5901,28.32,71.64,1004.73,72.71,436.31
...,...,...,...,...,...
2596,21.82,50.66,1013.74,73.60,455.20
4663,28.18,49.30,1003.97,55.56,438.91
3258,7.62,44.71,1020.56,69.96,486.07
1873,23.76,58.41,1013.85,85.38,445.85


In [108]:
utils.pickle_dump(train_set_cleaned[config["predictors"]], config["train_feng_set_path"][0])
utils.pickle_dump(train_set_cleaned[config["label"]], config["train_feng_set_path"][1])

utils.pickle_dump(valid_set[config["predictors"]], config["valid_feng_set_path"][0])
utils.pickle_dump(valid_set[config["label"]], config["valid_feng_set_path"][1])

utils.pickle_dump(test_set[config["predictors"]], config["test_feng_set_path"][0])
utils.pickle_dump(test_set[config["label"]], config["test_feng_set_path"][1])